### Notes - MLPP (newer unfinished edition)

- we want to learn $p(G|\mathcal{D})$ where $G$ is a graph structure, represented by a a $D$ by $D$ adjacency matrix.
- main problem: number of possible graphs is exponential in number of nodes
- the problem of graph structure learning for general graphs is NP-Hard.
- if the goal is knowledge discovery, we may want to compute the marginal of over whether an edge is present: $p(G_{st}=1|\mathcal{D})$
- if the goal is density estimation, we may want the MAP graph: $G^*=\textrm{argmax}_G p(G|\mathcal{D})$
- if we want to causal structure underneath the data, this is a much harder...
- 'quick and dirty' methods for learning graph structures which can be used to *visualize data*:
    - relevance network: way of viewing pairwise mutual infirmation. They tend to be very dense.
    - dependency netwrosk: fit $D$ sparse models to $p(x_t|\mathbf{x}_{-t})$ to determine its Markov Blanket. This approach doesn't add up to a consistent joint distribution though.
- learning *tree* structured graphs:
    - when we consider tree structure, undirected and direct models are the same - they can express the same distributions and have the same number of parameters and inference costs the same. Learning the graph structure is easier in a MN and parameter learning is easier for a BN.
    - Chow-Liu algorithm computes the ML tree structure. Edges weighted are pairwise mutual information between nodes. Chow-Liu than computes the maximum weight spanning tree using these.
    - We can use a penalized likelihood to find the MAP forest (for which inference is easier)
    - We can use a 'mixture of trees' where some joint observations are expected to come from a certain tree structure.
- Learning fully observed DAG structures: This is called Bayes Net Structure Learning. Here, we learn how to compute $p(G|\mathcal{D})$ where $G$ is a DAG structure.
    - Markov Equivalence: Two graphs (directed or undirected) are Markov Equivalent if they encode the same CI statements. With even an infinite amount of data, we may only learn graph structure up to Markov Equivalence. Don't read into edge directions too much!
        - Theorem: Two graphs are Markov Equivalent iff they have the same undirected skeleton and v-structures. We can represent a Markov Equivalence class with a PDAG.
    - If an algo is able to recover the PDAG with an infinite amount of data, we say it is a consistent estimator. For this to work, the generating distribution $P$ has to be **faithful** to the generating DAG. This means *all* the CI statements are captured by the graph structure: $I(P)=I(G)$ - this means there cannot be any CI statements that are due to particular settings of the parameters.
    - Let's assume everything is tabular and fully observed. Then there is a straight forward equation for a BN graph: $P(\mathcal{D}|G,\boldsymbol{\theta})$. The maximize of this as a function of $G$ and $\boldsymbol{\theta}$ will always be a fully connected graph. So we need to compute the marginal $P(\mathcal{D}|G) = \int P(\mathcal{D}|G,\boldsymbol{\theta})P(\boldsymbol{\theta})d\boldsymbol{\theta}$
    - If determine the prior $P(\boldsymbol{\theta})$, we can compute $P(\mathcal{D}|G)$.
        - Only a certain Dirichlet prior on the parameters gives you 'likelihood equivalence', where if $G_1$ and $G_2$ and Markov Equivalent, they have the same marginal likelihood.
    - In everything except CPTs and linear gaussian (which isn't discrete!), we need to *approximate* $p(\mathcal{D}|G)$.
    - Some methods for computing the mode of (or samples of ) $p(\mathcal{D}|G)$.
        - If we know the total ordering of the nodes, we can 'compute the distribution over parents for each node independently'.
        - We can use Dynamic Programming to find the globally optimal MAP DAG (see http://www.jmlr.org/papers/volume5/koivisto0
        4a/koivisto04a.pdf). We can also use DP to compute $P(G_{st}=1|\mathcal{D})$. They both take $D2^D$ time :(
        - There are *a lot* of DAGs for D nodes. As we increase D, the number of possible DAGs increases like: 3, 25, 543, 29281, 3781503
        - Hill Climbing, a greedy algorithm: 'At each step, the algorithm proposes small changes to the current graph, such as adding, deleting or reversing a single edge; it then moves to the neighboring graph which most increases the posterior. The method stops when it reaches a local maximum. It is important that the method only proposes local changes to the graph, since this enables the change in marginal likelihood (and hence the posterior) to be computed in constant time (assuming we cache the sufficient statistics).'
        - Sampling methods: If we want to compute $P(G_{st}=1|\mathcal{D})$, we can sample graphs and look at the fraction that have that edge.
        - Constraint-based approach: The IC (inductive causation) algorithm is a frequentist method. For pairs of nodes, we look for sets that make the CI. If we can't find them, we add a direction. We keep doing this for all pairs. Then we do some fanciness to determine v structures. The PC algorithm speeds this up by ordering the sets searched in a special way. This approach relies on an 'oracle' for determing a CI statement amongst three variables. The issue is.. these tests can be wrong and that wrongness can propagate throughout the rest of the graph. In practice, IC is used to create an initial structure to speed up Bayesian selection approaches.
        - The posterior mode (MAP) is known to converge to the MLE, which in turn will converge to the true graph G (up to Markov equivalence), so any exact algorithm for Bayesian inference is a consistent estimator. The IC/PC algorithm is consistent (given P is faithful.
- Learning DAG structure with latent variables:
    - We will discover that adding latent variables makes computing the marginal over a graph intractable. Basically, for each possible assignment of hidden variables, you have to integrate out all the parameters. Ouch!
    - The BIC$(G)$ is an estimate of the marginal likelihood $P(\mathcal{D}|G)$, though it often severely underestimates the mariginal likelihood.
    - Cheeseman-Stutz approximation: it's another approximation. Look it up for details
    - An even more accurate approach is the Varitional Bayes EM algorithm. There we make the approximation:
    $P(\boldsymbol{\theta},\mathbf{z}_{1:N}|\mathcal{D}) \approx Q(\boldsymbol{\theta})Q(\mathbf{z}_{1:N})  Q(\boldsymbol{\theta})\prod_i^N Q(\mathbf{z}_i)$. This ultimately provides a lower bound on $P(\mathcal{D}|G)$ that serves as a good approximate. It's between the Cheeseman-stutz!
    - Structure EM: One approach is to search over the space of graphs and use methods to estimation the marginal likelihood to pick graphs. The difficulty is.. in the presence of missing data.. the likelihood doesn't decompose, so it's very expensive for each search step. In structural EM, at the current graph and when considering candidate graphs, we use the current graph to fill in the missing data.. then with that filled in, we evaluate the neighboring candidate graphs as if the data is fully observed.
    - Discovering hidden variables:
        - Structure learning is based on the old neuroscience idea that "nodes that fire together should wire together".
        - "Recently, various methods have been developed that can recover the exact structure of the DAG, even in the presence of (a known number of) latent variables, under certain assumptions."
- Learning Undirected Graph Structures: In this section, we discuss how to learn the structure of undirected graphical models. On the one hand, this is easier than learning DAG structure because we don’t need to worry about acyclicity. On the other hand, it is harder than learning DAG structure since the likelihood does not decompose (see Section 24.6.1). This precludes the kind of local search methods (both greedy search and MCMC sampling) we used to learn DAG structures, because the cost of evaluating each neighboring graph is too high, since we have to refit each model from scratch (there is no way to incrementally update the score of a model)
    - Graphical Lasso - which is for continuous variables.
    - Though: 'It is possible to extend the graphical lasso idea to the discrete MRF and CRF case.'
    - In the continuous case, there is one parameter per edge, which makes things simple. In the discrete case, there is a set of parameters. So since we used the Lasso in the continuous case, we have to use the GROUP lasso (discrete analogy) in this case.

### PGM - 18 - Structure Learning in Bayesian Networks

- We will assume in this chapter:
    - the data is fully observed
    - $P^*$ is 'induced' by some BN $\mathcal{G}^*$.
- In an empirical distribution, we don't expect to find the *exact* CI relations to hold.
- Its hard to tell whether what you observe the result of true independence or a weak connection
- Reasons for learning the model structure:
    - Knowledge discovery - maybe we'd like to know whether two things are related?
    - Density estimation - here we care about generalization to new instances
- We can only recover the equivalence class of $\mathcal{G}^*$
- We should anticipate errors in stating whether edges exist. We have to consider the tradeoffs when we declare too many edges (too few CI statements) or too few edges (too many CI Statements). Balancing this needs to be done in the context of *how* we intend on using these PGMs.
- If we care about generalization, then we probably want to have *too few* edges. This is because we will have greater certainty regarding the parameters we learn (because there are fewer of them). That certainty is very important for generalization. The reason for this certainty is due to *data fragmentation*. With more edges, you fragment the data more - such that there are fewer data pointers to attest to each parameter.
- There are 3 methods:
    - constraint-based structure learning: we use statistical tests to directly capture CI statements. We will not focus on this because its very sensitive to errors
    - score-based structure learning: We consider each graph structure a separate *model* and we think of this as model selection problem. In fact, we define a *hypotheisis space* of potential models (these aren't necessarily 1-to-1 with a graph though.. -wait.. is this true??) and then search this space. This space is huge, so we tend to use greedy methods to search it. We prefer some over the other with a yet to be defined score function.
- Constraint-Based Approaches: SKIP
- Structure Scores:
    - Likelihood Scores: The score of the graph is the likelihood of the data according to the MLE parameters. These end up being Entropy and MI measures. The difference in scores (which is used to pick one graph over another) has a nice amount of cancellation. This will prefer networks where the parents of a node carry a lot of information on that node. The problem is.. the score *always* prefers more complicated networks.. extra parameters (almost surely in the limit) always make the data look more like. We will get more complex models - poor generalization!
    - Bayesian Score:
        - Philosophy: If we are uncertainty regarding anything, we should put a distribution around it.
        - We use: $P(\mathcal{D}|\mathcal{G})=\int_{\Theta_\mathcal{G}}P(\mathcal{D},\boldsymbol{\theta}_\mathcal{G}|\mathcal{G})P(\boldsymbol{\theta}_\mathcal{G}|\mathcal{G})d\boldsymbol{\theta}_\mathcal{G}$
        - It's like an likelihood score, but it's a weighted average according to our prior $P(\boldsymbol{\theta}_\mathcal{G}|\mathcal{G})$
        - This is crazy: Imagine we used a holdout set that we use to evaluate a structure.. In a way (see the text), the marginal likelihood is computing the same thing! Except without the loss of the holdout data. This suggestions:
        
        $\frac{1}{M}\log P(\mathcal{D}|\mathcal{G})\approx\mathbb{E}_{P^*}[\log P(\mathcal{X}|\mathcal{G},\mathcal{D}]$
        
        For reasonable size samples, this intuition holds.
        - The marginal likelihood will favor simpler models.. only as the data gets larger will it consider more complex model.
        - If the parameters are satisfy 'global parameter independence', that means they are apriori independent. With that, the Bayesian Score decomposes into products of (sum of in log space) sub problems.. one involve each family (a child node and all it's parents). Which is very useful.
        - In fact, 'Considering more general structures, if $Pa_{X_i}^\mathcal{G}=Pa_{X_i}^{\mathcal{G}'}$, then it would seem natural that the term that measures the score of $X_i$ given its parents in $\mathcal{G}$ would be identical to the one in $\mathcal{G}'$. This seems reasonable. Recall that the score associated with $X_i$ measures how well it can be predicted given its parents. Thus, if $X_i$ has the same set of parents in both structures, this term should have the same value.' 
        - This means the Bayesian Score is *decomposable* (actually, only if the parameters for a node under graphs with the same set of parents for that node have the same probability). A score is this way if it can be written:
        
        $SCORE(\mathcal{G}:\mathcal{D})=\sum_i \textrm{FamScore}(X_i|Pa_{X_i}^\mathcal{G}:\mathcal{D})$
        - This is important, because when we perterb the graph in a search, many of the terms want change. So we will save ourselves a lot of compute.
    - BIC:
        - As the data grows, $M \rightarrow \infty$, the BIC approximation (to the Bayesian Score) holds better:
        
        $\log P(\mathcal{D}|\mathcal{G}) \rightarrow l(\hat{\boldsymbol{\theta}}_\mathcal{G}:\mathcal{D}) - \frac{\log M}{2} Dim[\mathcal{G}] + O(1) = SCORE_{BIC}(\mathcal{G}:\mathcal{D})$
        
        We can decompose it further:
        
        $
        SCORE_{BIC}(\mathcal{G}:\mathcal{D}) = M\big(\sum_{i=1}^n \mathbb{I}[X_i|Pa_{X_i}] -  \sum_{i=1}^n\mathbb{H}(X_i)\big) - \frac{\log M}{2} Dim[\mathcal{G}]
        $
        
        The score exhibits a trade-off between fit to data and model complexity: the stronger the dependence of a variable on its parents, the higher the score; the more complex the network, the lower the score. However, the mutual information term grows linearly in M, whereas the complexity term grows logarithmically. Therefore, the larger M is, the more emphasis will be given to the fit to data.
    - The Bayesian Score and the BIC are consisten. Meaning in the limit, they recover the equivalence class of $\mathcal{G}$.
    - The prior $P(\mathcal{G})$ doesn't have a big impact. It's mathematically convenient for it to have structure modularity, which mean its a product of terms, one for each familiy:
    $
    P(\mathcal{G}) \propto \prod_i P(Pa_{X_i} = Pa_{X_i}^\mathcal{G})
    $
    It's also reasonable to suggest that if two graphs are I-equivalent, they get the same prior probability (and posterior too right?)
    - If you want score equivalence (which means two graphs that are I equivalent get the same score), you must use the DBe prior (if you also want decomposition property and  Dirichlet prior on paramers).
- Structures Search:
    - Our inputs are some $\mathcal{D}$, scoring functions which evaluate a network, and a set of possible network structures.
    - Our desired output is a network structure that maximizes the score
    - The important properties that we want our score to have our decomposability and score equivalence.
    - It is computationally efficient to learn tree structured graphs. A tree structure graph is one where each node has at most one parent.
    - Another easy case is when there is a known ordering to the variables, so that for any edge, we know which one would be the parent and which one be the child. This isn't as unrealistic as it sounds. When we have a temporal element, this can work.
    - For general graphs, we have a complication to consider. If we decide to add an edge, that may constrain us down the line. Specifically because we can't have any directed cycled.
    - We have this sad theorem: Given a decomposable score, finding the argmax network to the score is an NP-Hard.
    - With that, we will use heuristic that aim to find the best network, but aren't guaranteed to do so.
    - 'We solve this problem using a local search approach. To do so, we define three components: a search space, which defines the set of candidate network structures; a scoring function that we aim to maximize (for example, the BDe score given the data and priors); and the search procedure that explores the search space without necessarily seeing all of it (since it is superexponential in size).'
    - Think of the 'search space' as a (different) graph over candidate solutions. A node/solution is a graph (most of the time - not always though). An edge moves you from one solution to another with a single 'operation'. There are 3 obvious operators:
        - add an edge
        - delete an edge
        - edge reversal
    - The consideration of the number of neighbors is important. If nodes have a small number of neighbors, evaluating the next move is fast.. because we don't need to consider that mean neighbors (don't need to evaluate the score function). However, it may take you many steps to get to your solution. 
    - The good thing about the 3 operators we mention is that they imply low diameter in the space. It takes you at most $n^2$ operations to go between any two networks.
    - Also, edge addition/deletion can only effect one family score, so all others will remain same. a reverse will effect two families. The point is.. from one node to the next.. most terms in the score stay the same.
    - (The explain why deletion/reversal are necessary.. in effect, they are there to undo bad choices made earlier in the search)
    - The search is generally some form of greedy algorithm.
    - The cost: When we move from one node to the next, we need to consider $O(n^2)$ operations. If we take $K$ steps, this means the whole things costs $O(Kn^2)$.
    - Problems with search:
        - Local Maxima: all neighors reduce the score
        - Plateau: neighors for a long stretch all have the same score. We expect to comes across these. The I-equivalence class of networks will all produce the same score.. and you can get from one to the other with edge reversals. If we come across an equivalence class, either:
            - From one of the nodes in the equivalence class.. you can make a change to improve the score
            - The whole class is a local maxima
        - Solutions to Plateau's:
            - enumerate the whole class and check all neighbors of each. This can be expensive.
            - Change the search space such that edge node is an equivalence class
    - tabu search: make sure you don't undo the operations you've made in the past L steps.
    - Randomization is helpful to escape local maxes.
    - Data Pertubation Methods:
        - When we come across local maxima, we can perturb (actually, re-weight) our data such that it is no longer a local maxima across to our score function.
    - Sec 18.4.3.3 - they talk precisely why decomposability let's us evaluate scores faster (which is the big cost in this)
    - Learning with Equivalance Classes:
        - We can use a PDAG to represent an equivalence class
        - To score a PDAG, we score *any* network in that class and score that. By score equivalence, this will be the score of that PDAG.
        - What about the operations?
            - We can consider PDAG specific changes (we will need more operations since it has both direction and undirect edges)
            - We can use operations in DAG space that are guananteed to move us across equivalence classes.
        - The algorithm that does this is the GES algorithm. It works reasonable well empirically and has solid theoretical properties.
        - There is additional cost from this:
            - Traversing equivalence classes is more complicated/expenses.
            - Reusing fam score stuff is harder.
        - These have been addressed to some extent, and that's why this algo works.
        
        


### PGM - 20.7 - Structure Learning (Undirected Models)

- There are two approaches:
    - constraint-based approaches: search for a graph structure satisfying the independence assumptions that we observe in the empirical distribution
    - define an objective function for different models, and then search for a high-scoring model.
- Advantages of the constraint based approach:
    - CI statements of a MN are simplier
    - score functions require evaluating the likelihood, which is expensive for a MN.
- Disadvantages:
    - relies heavily on tests of statistical independence, which aren't necessarily robust to statistical noise in the empirical distribution.
    - It only gives us a graph/network structure. Not a fully specified model. So we still need to do parameter learning, where we will hit the cost of that likelihood evaluation.
    - It's not clear that learning the CI statement is what we want. Maybe it's important to learn the factors/features directly. As an extreme example, imagine a Gibbs distribution characterizes by every node being involved in a pair wise factor with every other node. The graph implied is fully connected (which isn't useful), but the pairwise factors are!
- Score based approaches get the focus because of these reasons.
- Structure Learning Using Independence Tests
    - Assumes:
        - generating distribution $P^*$ is positive
        - $P^*$ can be represented as a MN graph $\mathcal{H}^*$ which is a *perfect map.* 
        - The degree of nodes in $\mathcal{H}^*$ is at most $d^*$
    - There are 3 types of independence statements: pairwise, markov blanket vs everything else and global statements. In positive distributions, these are equivalent.
    - We want to run as few statistical tests as possible, so we should consider pairwise statements and the markov blanket statements.
     - Pairwise:
         - $(X \perp Y | \mathcal{X} - \{X,Y\}) \ \  \forall (X-Y) \notin \mathcal{H}$
     - Markov:
         - $(X \perp \mathcal{X} - X - MB_{\mathcal{H}^*}(X)|MB_{\mathcal{H}^*}(X)) \ \  \forall X$
     - As stated, these are *very* hard to test for. 
     - Let's just poop on this idea and move on. We will not review it
     
 - Score-Based Learning:
     - We define a hypothesis space as a space of possible graphs. We intent to search this space. Each graph will recieve a score that we will use to evaluate it.
     - There are levels of granuality that we can consider network parameterization:
         - Coarsest level: The MN graph and measure complexity in terms of the size of the cliquies
         - More granular: We consider the size of factors as complexity.
         - Most granular: Consider (indicator?) features in a log-linear model and measure sparsity at the level of features included.
     - The more fine-grained our hypothesis space, the better we will be position to learn a parameterization. 'For example, the factor-graph approach allows us to distinguish between a single large factor over k variables and a set of  $k \choose 2$ pairwise factors over the same variables, requiring far fewer parameters. The feature-based approach also allows us to distinguish between a full factor over k variables and a single log-linear feature over the same set of variables.'
     - 'Conversely, the finer-grained spaces can obscure the connection to the network structure, in that sparsity in the space of features selected does not correspond directly to sparsity in the model structure.' For example, if you decide the feature $f(\mathbf{d})$ exists, then all the variables of $\mathbf{D}$ are included. This means a small number of features can yield a densely connected graph. Finer grained networks also have large spaces to search over, so that can slow things down. This can make down the line inference harder.
     - DID NOT FINISH
     


### Answer structure

Things worth mentioning:
- Why we *shouldn't* do constraint based structure learning
- What are our assumptions?
- The generic structure of searching: the search space, the nodes, the operators
- The equivalence class of a graph (and PDAGs?)
- Good properties of a score:
    - Score equivalence
    - Decomposable
- The Bayesian Approach:
    - The Bayesian Score
    - BIC
    
Outline:
- Intro
- What is the task?
- How would you do it? Structure learning? Doesn't work!
- The generic approach: searching a space over network structures
- 

### How is the Graph Structure of a PGM learned?

(copy line about 'this is a series' here)

*Everything* up until now has been from the starting point of 'Given a graph of nodes and edges...'. Here, we'll learn how to determine that graph. With that, we clear the final hurdle in the daunt tasking PGMs address: to understand a generic complex system. 

However, this task is, in a sense, the hardest. Abstractly, it is responsible for integration over a space much larger than anything we've encountered so far. Because of this, this matter contains large unsolved or intractable pockets. On the bright side, we'll see an important scientific boundary - one that separates the doable from the challenges of Machine Learning.

But, as always, let's start with a refresher.

### Refresher

Things to cover here:

- What is a complete system
- What is a PGM
    - Need to go into further detail about CI Statements
- A BN
- A MN
- Maybe: MN learning requires inference?
- Maybe: The likelihood function?

### What are we assuming and what's the goal?

We start by assuming we have fully observed data $\mathcal{D}$, which is $w$ observations of all RVs of $\mathcal{X}$. Later, we'll discuss what happens when we don't have fully observed data. 

Our goal depends on whether we're designing a BN or an MN. In the case of a BN, we seek to learn a DAG - a directed acyclic network that connects our RVs. In the case of an MN, we'll seek to learn an undirected graph. We call this **structure learning.**

With these determined, we can proceed learning their parameters according to the tactics we've learned in answers 5 and 6. With those determined, we can understand our complex system with the inference methods learned in answers 2, 3 and 4.

Though I should provide a caveat. As clean as I'm making this separation appear, the reality is in fact more complicated. That reality is that approximations are almost certainly necessary at some point and the best choice of those approximations depend on our downstream use of the PGM. This creates a dependency amongst our choices made at structure learning, parameter learning and inference. In light of this fact, some methods straddle these separations quite deliberately. Regardless, the separation of these stages is a useful way to conceptual the task of developing a PGM.

### A BN or an MN?

Presumably, we start this process by staring at a big lump of observations $\mathcal{D}$. Our first question is, do we want to use a BN or an MN? Typical, this choice is *pure judgement.* To help with that, I'll reguritate something from answer 1:

"
Broadly, MNs do better when we have decidedly associative observations - like pixels on a screen or concurrent sounds. BNs are better suited when we suspect the RVs attests to distinct components of some causal structure. Timestamps and an outside expectation of what's producing the data are helpful for that.
"

That said, the choice is usually clear from our qualitative understanding of the system. Hence we are fairly confident in this choice. So going forward, let's assume we can make this choice.

(Transition)

### The obvious solution that doesn't work.

Recall from the refresher that, a graph, whether its for a BN or an MN, implies a set of CI statements regarding $\mathcal{X}$. An irresistible solution is to simply ask of $\mathcal{D}$ which CI statements are true and then generate the graph that reflects those CI statement[1]. As appealing as this sounds, such approaches don't work well for two reasons:

1. Since $\mathcal{X}$ is a mere sample from the true distribution $P(\mathcal{X})$, we never expect CI statements to hold *exactly* in our data. Therefore, we are forced to rely on some statistical test to determine whether a given CI statement is true. Such tests are undoubtedly subject to error. This is an issue because as we grow our set of CI statement, errors propagate - false statements in the past make us more likely to make false ones in the future.

2. It is likely that we need to invoke these statistical test many times. Since they are designed to admit a controlled but non-neglible percentage of errors, as the number of invocations grows, so does the number of errors.
    
Because of this, we will not explore such solutions. Though they aren't useless. For example, they can serve as useful initialization strategies for other approaches.

Instead, we will consider that which works best in practice. 

### Score-Based Structure Learning

The idea is to *search* a 'space' of candidate graphs and to select the best one according to some score. To execute this, we must specify four components of this strategy:

1. The **Search Space** of candidate graphs over $\mathcal{X}$: Choosing this will determine what we think matters in differentiating graphs. The simplest and most typical search space is simply the space of graphs themselves. So if two graphs occupy the same point in that space, they have the same set of edges.[2] Let's call this the 'standard graph' space. However, it doesn't *have* to be this way. For example, you could have a (rather silly) search space over the *count* of edges in the graph. So two graphs would occupy the same point if they have the same number of edges. [3]

2. A set of **operators**: Applying one of these once *defines* how to move from one point in the search space to a *neighboring* point. Let's say that differently. If we are sitting at a point (graph) in our search space, any point (graph) that can be reached by applying a *single* operator *one time* is considered a *neighbor*. For example: if we consider the standard graph search space for a BN, the three most common operators are **edge addition**, **edge deletion** and **edge reversal**. With this, you can think of our search space as a graph itself - nodes are our points in the search spaces and edges connect two points that can be reached with one operation.

3. The **Score Function**: This is a function that takes a point in our space and tells us, with one number, how good that point/graph is. Unsurprisingly, this almost always involves a likelihood function. So, a graph that does well makes our observed data very likely.

4. A **Search Algorithm**: This is an algorithm that accepts components 1-3 and dictates how we explore the space and ultimately return our best answer. For example, the simplest algorithm might accept some starting point, generate all the neighboring points using our operators, score each one and move to the point that achieved the highest score. It would repeat this until the scores stop improving and then return the final point.

As is probably clear, components 1-2 are very much entangled - you cannot decide one without considering the other. Also, the score function heavily depends on the choice of search space, since it must accept a point in that space. In general, most score functions expect a point from the standard graph space. The last component is technically agnostic of 1-3, but it's practical effectiveness isn't. For example, if the score function is very expensive to evaluate, the best search algorithm will call that function fewer times.

Now, let's specify all these with the task of:

### Learning the structure of a Bayesian Network

First, make a mental note that we just limited the task to a BN. Later, we'll consider an MN.

Now, step one is to determine our space over graphs. To keep things uncontroversial, let's use the standard graph search space. A natural question is: how big is this space?

The answer becomes clear if we use an $n$-by-$n$[4] **adjacency matrix**, which contains only 1s and 0s, to represent a graph. In the case of a *directed* graph, this matrix has a 1 at row $i$ and column $j$ only if the graph contains the the edge $X_i \rightarrow X_j$. Now, this representation includes graphs with cycles, which are illegal in BNs. Regardless, we can see there are $2^{\mathcal{O}(n^2)}$ graphs. To put it mildly, that's too many to consider. To put it accurately, our search algorithm will be traversing millimeters in intergalatic space. You only need... $n=12$ for that to hold![5]

Next, let's go with our previously mentioned operators: edge addition, deletion and reversal.

Now for the hard part - the score function. Before I provide one, we should first cover what we want out of it.

### Properties of a good score function for BN structure learning.

Before dividing in, let's set up some notation. Let's use $\mathcal{G}^0$ to denote any input directed graph. We'll use $\textrm{Score}(\cdot)$ to refer to our score function. It's not explicit here, but this score function is a function of our data - so keep that in mind.

Now, there are certain properties we'd like in our $\textrm{Score}(\cdot)$.

1. **Consistency**: If we were to sample an infinite amount of data, our score function would assign it's max value to the true graph structure. For this to be possible, the true graph needs to be **faithful** to some DAG. In other words, the true distribution must have a set of CI statements all of which are perfectly represented with some DAG. Goin forward, we'll assume this is case.
2. **Score Equivalence**: To describe this, we need to mention **Markov Equivalence**. Two graphs are Markov equivalent if they imply the same set of CI statements. Perhaps surprisingly, two different directed graphs can imply the same CI statements. It turns out that two directed graphs are Markov Equivalent if they have the same 'skeleton' and 'v-structures.' The skeleton of a directed graph is the undirected graph created by converting all edges to undirected edges. A v-structure is where we have 3 nodes/RVs, say $A$, $B$ and $C$, whereby $A\rightarrow B \leftarrow C$. To put it simply, v-structures are the only orientation of edges that make a difference in the CI statements, so we need to keep track of them. This is important because if two graphs are equivalent, then we could pick their parameters such that they yield the exact same distribution over $\mathcal{X}$. Not cool! Differences in the likelihood are basically our only way of picking between two options. So this means two graphs that are Markov Equivalent *should* get the same score, because they can always explain the data equally well. This is called Score Equivalence.
3. **Decomposable**: A score for a BN graph is decomposable if it is a sum of terms, each of which is a score of a particular 'family' (a child node plus it's parent nodes). If this is the case, when you change a graph with one operation, only one or two family scores change, so you don't have to recompute most of the terms when you calculate the next score. This property may be written:

$$
\textrm{Score}(\mathcal{G}^0) = \sum_j^n \textrm{FamScore}(X_j|Pa_{X_j}^{\mathcal{G}^0})
$$

So in short, we want a Score that will recover the Markov equivalence class (because that's best we can hope for) of the true graph in the limit of infinite data, assign equal values to all graphs in that class and to be easily recomputable after making small changes to the input graph.

This is a tall order. Who will help us?

The Bayesians.

### A Score Function for BN graph learning.

Those crafter thinkers say that the score should tell us how likely that graph is. That is:

$$
\textrm{Score}(\mathcal{G}^0) = P(\mathcal{G}^0|\mathcal{D})
$$

where $P$ refers to the 'true' distribution. So the Bayesians are telling us *what* we want, but not how to compute it.

Ok, now let's think downstream a bit - to the search algorithm. All the score function needs to do is tell us which graphs are *better* than others, so we know where to travel next. So we don't actually need the value $P(\mathcal{G}^0|\mathcal{D})$, but rather the rank ordering of graphs according to those values.[6] The lesser demand means we can trim some fat off this task.

First, taking the log of our scores won't change their ranking, so we apply that to simplify our life. Next, we sprinkle in a little Bayes Theorem:

$$
\begin{align}
\log P(\mathcal{G}^0|\mathcal{D}) & = \log \frac{P(\mathcal{D}|\mathcal{G}^0)P(\mathcal{G}^0)}{P(\mathcal{D})} \\
& = \log P(\mathcal{D}|\mathcal{G}^0) + \log P(\mathcal{G}^0) - \log P(\mathcal{D}) \\
\end{align}
$$

Oh look! $\log P(\mathcal{D})$ is the same for all graphs, so we can ignore it! Awesome - now we can write the score function as:

$$
\textrm{Score}(\mathcal{G}^0) = \log P(\mathcal{D}|\mathcal{G}^0) + \log P(\mathcal{G}^0)
$$

Let's consider these terms. The first obvious observation is that we don't have nature's true distribution, so we'll have to use a model for it. The $\log P(\mathcal{G}^0)$ term is the prior-to-seeing-the-data probability of this graph. Without seeing the data, what can you really do? Well, we can say all graphs are equally likely, in which case we can drop it, since it won't affect our ranking. Or we could prefer less complicated graphs (those with fewer edges).

Either way, the action is in $\log P(\mathcal{D}|\mathcal{G}^0)$. This, we can handle with the tools we've developed.

First, we've been designing a PGM to approximate nature's $P$, so let's use $\log P(\mathcal{D}|\mathcal{G}^0)\approx \log P_B(\mathcal{D}|\mathcal{G}^0)$. That is, we'll use the probabilities produced by our BN to approximate reality.

Next, we realize that we know how to calculate the likelihood of our data given a graph *and* an associated set of parameters. We've done this before in the **likelihood function.** Since that dependency is important here, let's emphasize that in our notation. Let's say $P_B(\cdot|\mathcal{G}^0,\boldsymbol{\theta}_{\mathcal{G}^0})$ is our BN probability distribution (generated via the Chain Rule) given the graph $\mathcal{G}^0$ and the parameters associated with that graph, $\boldsymbol{\theta}_{\mathcal{G}^0}$. So, take my word that we know how to calculate $P_B(\mathcal{D}|\mathcal{G}^0,\boldsymbol{\theta}_{\mathcal{G}^0})$. If you don't, see answer 5 [link].

At this point, we call up those genius Bayesians and they tell us how to use this to get $P_B(\mathcal{D}|\mathcal{G}^0)$:

$$
P_B(\mathcal{D}|\mathcal{G}^0) = \int_{\boldsymbol{\Theta}_{\mathcal{G}^0}} P_B(\mathcal{D}|\mathcal{G}^0,\boldsymbol{\theta}_{\mathcal{G}^0})P_B(\boldsymbol{\theta}_{\mathcal{G}^0}|\mathcal{G}^0)d \boldsymbol{\theta}_{\mathcal{G}^0}
$$

This may look intimidating, but it doesn't deserve such a reaction.

First, we notice there is one new raskel: $P_B(\boldsymbol{\theta}_{\mathcal{G}^0}|\mathcal{G}^0)$. We haven't touched this before explicitly. To produce such a quantity, our BN must include the parameters themselves as RVs. In other words, we add them to $\mathcal{X}$. Though, they are never observed - they are **hidden variables**. The 'prior' distribution governing these are typically decided exogenously. If we are using table CPDs, a common choice is to use a Dirichlet distribution[link] for chunks of $\boldsymbol{\theta}_{\mathcal{G}^0}$. Those chunks are those that determine the distribution of a child node given a certain assignment of its parents.

Now we can think about that expression. All it is is an average of our likelihoods, $P_B(\mathcal{D}|\mathcal{G}^0,\boldsymbol{\theta}_{\mathcal{G}^0})$, weighted by the prior probability of the parameter values for each likelihood. This average is taken over the entire space of parameters, $\boldsymbol{\Theta}_{\mathcal{G}^0}$.

And, uh... that last line is a big problem. If we have many parameters, there is typically no chance we'll be able to perform such an intraction. As such, we resort to approximations - like Monte Carlo simualtions. 

But before addressing that headache, let's soak in some good news: If we use $\textrm{Score}(\mathcal{G}^0)=\log P(\mathcal{D}|\mathcal{G}^0)$, then this score is consistent, score equivalent and decomposable. We can maintain these properties if we add in $\log P(\mathcal{G}^0)$, as long as that prior distribution obeys similar properties. This is called the **Bayesian Score**, which we'll denote with $\textrm{Score}_{BS}(\cdot)$.

Now, let's reboot our headache - $\boldsymbol{\Theta}_{\mathcal{G}^0}$ is too large/expensive to integrate over. So now what? Well, some clever mathematicians have told us that, if we use a certain reasonable prior, then:

$$
\begin{align}
\textrm{Score}_{BS}(\mathcal{G}^0) & \rightarrow \log P_B(\mathcal{D}|\mathcal{G}^0,\hat{\boldsymbol{\theta}_{\mathcal{G}^0}}) - \frac{\log w}{2}\textrm{Dim}(\mathcal{G}^0) + \textrm{const}\\
\textrm{as } w & \rightarrow \infty
\end{align}
$$

where $\textrm{Dim}(\mathcal{G}^0)$ is the 'dimension' or 'degrees-of-freedom' of $\mathcal{G}^0$. In the case of a BN, this number is well approximated by the number of parameters of $\mathcal{G}^0$. Also, $\hat{\boldsymbol{\theta}_{\mathcal{G}^0}}$ are the MLE parameters according to $\mathcal{G}^0$ and $\mathcal{D}$.

So this suggess a new score, called the **Bayesian Information Criteria**:

$$
\textrm{Score}_{BIC}(\mathcal{G}^0) = P_B(\mathcal{D}|\mathcal{G}^0,\hat{\boldsymbol{\theta}_{\mathcal{G}^0}}) - \frac{\log w}{2}\textrm{Dim}(\mathcal{G}^0)
$$


$$
\textrm{Score}_{BIC}(\mathcal{F}^0) = P_B(\mathcal{D}|\mathcal{F}^0,\hat{\boldsymbol{\theta}_{\mathcal{F}^0}}) - \frac{\log w}{2}\textrm{Dim}(\mathcal{F}^0)
$$

Now, this - this is tractable. Also, it maintains those properies we love so much. So let's say this is our score function.

Finally, we hit the final component.

$$
\hat{\boldsymbol{\theta}_{\mathcal{F}^0}}
$$

### The search algorithm for learning a BN structure.

All the remains to decide is an algorithm that accept our first three choices, explore the space and return the best point. Here is a dirt simple example algorithm:

1. We provide the algorithm with a starting point, the operators and a score function.
2. The algorithm applies the operators to the starting point, yielding a set of neighboring points.
3. We apply the score function to each of the neighboring points. We move our current point to the neighbor which yields the highest score.
4. Repeat until scores stop improving.

As simple as it is, this is perfectly reasonable. We now have a workable means to learning the graph structure of a BN.

That said, we can anticipate some issues:

1. This algorithm is defendless against local optima. At the very least, we should run this algorithm in parallel, supplying each run with randomly initialized graphs.
2. If an operation yields a neighbor that is in the same Markov Equivalence class as the current point, we *know* the score will be the same. So it's a waste of time to compute that score. Therefore, we should make sure our operation cross equivalences. This is harder than it sounds.

Given that last point, maybe a point in our search space *shouldn't* imply a single set of edges, but rather an entire equivalence class. Granted we found operators that could traverse such points, we'd solve that problem.

As you can tell, we can get quite fancy in choosing these components.

But instead, let's move onto learning a Markov Network. As we will discover, this shares many of the same components as we've seen, just mixed with bad news.

### Learning the strucuture of a Markov Network.

Just like it did in learning the parameters of a MN, it'll help to restrict attention to the **log-linear** form of an MN. This was descriped in answer 6:

(copy that chunk from answer 6)

Now, from here, we're much better off considering a search that *isn't* the standard graph space. This is due to the fact that once you have the graph learned, there remains a gap before learning parameters begins. Particularly, imagine you learned the graph and it's fully connected. Now, we ask: What subsets of RVs does each feature map assignments from? At one extreme, it could be a single feature that accepts an assignment to all of $\mathcal{X}$. Or, it could be $n \choose 2$ features that map from every connected pair of RVs. The point is, there is a choice that remains betwee knowing the graph and learning their parameters.

Because of this, MN structure learning tends to involve a more granular search space. One where a point will land us, at least, at the doorstep of parameter learning.




$$
\textrm{Score}_{BIC}(\mathcal{H}^0) = P_B(\mathcal{D}|\mathcal{H}^0,\hat{\boldsymbol{\theta}_{\mathcal{H}^0}}) - \frac{\log w}{2}\textrm{Dim}(\mathcal{H}^0)
$$

### What happens with missing data?

### Footnotes

[1] Actually, as stated, this wouldn't work. Even if we could determine all the CI statements, it's quite possible *no graph* could represent the CI statements we get.

[2] Just edges! They can't have a different number of nodes because all graphs must have a node for each RV in $\mathcal{X}$.

[3] You might reasonably complain here: "Hey! That silly space isn't one-to-one with graphs. If you find the best point, you still have to consider a bunch of graphs!" Yes, that space is stupid. But it does highlight that you don't need this one-to-one-ness. To grossly oversimplify, sometimes just having the point (and not a specific graph) still accomplishes what we need in the end.

[4] Remember, $n$ is the number of RVs in $\mathcal{X}$.

[5] Assuming an edge in the search space is a millimeter and there are 6 million light years between galaxies on average.

[6] Ok, I'm sorry - this isn't exactly true. Some search algorithm use more than just the rank ordering of candidate graphs. Often, they utilize the magnitude of their differences as well. However, that magnitude is primarily driven by what remains after we trim that fat. Using just the rank ordering makes the presentation cleaner. 

### Scrap

### How should we think about a graph?

For a single graph, it's fair to picture it as we've seen: just nodes representing the RVs of $\mathcal{X}$ along with directed or undirect edges between them.

However now, we must consider many of these graphs, so we may select the best one. Therefore, we will consider the 'space' of all possible graphs for $\mathcal{X}$. As intimidating at this sounds, it's quite easy. We use an $n$-by-$n$[1] **adjacency matrix** which contains only 1s and 0s to represent a graph. In the case of a *directed* graph, this matrix has a 1 at row $i$ and column $j$ only if the graph contains the the edge $X_i \rightarrow X_j$. In the case of an *undirected* graph, this matrix has a 1 at row $i$ and column $j$ only if the graph has an edge between $X_i$ and $X_j$. As you can imagine, in the undirected case, this matrix is symmetric. With this view, it's easy to see that for directed and undirected graphs, there $2^{n^2-n}$ and $2^{(n^2-n)/2}$ possible graphs respectively. That's a lot of graphs!

Now, we explore:

-----------------
----------------------

As previously mentioned, how 'good' a graph is how likely it makes our data $\mathcal{D}$. It's exactly the same idea for the **likelihood function** mentioned in answers 5 and 6[link]. With that, how would design $\textrm{Score}(\mathcal{G}^0)$?

Here is an intuitive idea. The score function uses the data to learn a set of parameters according $\mathcal{G}^0$. Let's call those parameters $\hat{\boldsymbol{\theta}_{\mathcal{G}^0}}$. Then we return the log likelihood of our data according to those parameters and $\mathcal{G}^0$. So that is:

$$
\textrm{Score}(\mathcal{G}^0) = \log \mathcal{L}_{\mathcal{G}^0}(\hat{\boldsymbol{\theta}_{\mathcal{G}^0}})
$$

This is simple, but suffers a fatal flow. More complicated graphs, those with more edges, have more parameters and therefore, can *always* explain the data better. The score has no penalty for complexity, so it'll always return the most complicated graph - a fully connected graph!

Though this idea guides us directly to the most preferred solution. That solution comes from those productive Bayesians. They suggest that what we *really* want the score to tell us is: the probability of $\mathcal{G}^0$ is the true graph given the data:

$$
\textrm{Score}(\mathcal{G}^0) = P(\mathcal{G}^0|\mathcal{D})
$$

Recall, $P$ refers to the 'true' distribution. So the Bayesian are telling what we *want*, but not how to compute it.

Now, if we sprinkle in some Bayes Theorem, we get:

$$
P(\mathcal{G}^0|\mathcal{D}) = \frac{}{P(\mathcal{D})}
$$

---------------


If we say $\mathbf{X} = \mathcal{X}$, then we can write our data as $\mathcal{D}=\{\mathbf{x}^{(i)}\}_{i=1}^w$. Now we write out the likelihood(which we've seen before in answer 5[link]):

$$
P_B(\mathcal{D}|\mathcal{G}^0,\boldsymbol{\theta}_{\mathcal{G}^0}) = \prod_{i=1}^w \prod_{j=1}^n P_{B}(X_j=x^{(i)}_j|\textrm{Pa}_{X_j}^{\mathcal{G}^0}=\mathbf{x}^{(i)}_{\textrm{Pa}_{X_j}^{\mathcal{G}^0}},\boldsymbol{\theta}_{\mathcal{G}^0})
$$

I realize this notation is getting a little heavy, but realize: all we are doing is calculating the probability of our data according to a BN graph and some associated parameters. This is familiar stuff!

Next, we call up those genius Bayesians and they tell us how to use this to get $P_B(\mathcal{D}|\mathcal{G}^0)$:

$$
P_B(\mathcal{D}|\mathcal{G}^0) = \int_{\boldsymbol{\Theta}_{\mathcal{G}^0}} P_B(\mathcal{D}|\mathcal{G}^0,\boldsymbol{\theta}_{\mathcal{G}^0})P_B(\boldsymbol{\theta}_{\mathcal{G}^0})d \boldsymbol{\theta}_{\mathcal{G}^0}
$$

This may look intimidating, but it doesn't such a reaction.

--------------------

Next, we realize that the **likelihood function** mentioned in answers 5 and 6[link] gives us the pieces to construct this. To see this, let's think - we know how to calculate the likelihood of our data given a graph *and* an associated set of parameters. Since that dependency is important here, let's emphasize that in our notation. Let's say $P_B(\cdot|\mathcal{G}^0,\boldsymbol{\theta}_{\mathcal{G}^0})$ is our BN probability distribution (generated via the Chain Rule) given the graph $\mathcal{G}^0$ and the parameters associated with that graph, $\boldsymbol{\theta}_{\mathcal{G}^0}$.

If we say $\mathbf{X} = \mathcal{X}$, then we can write our data as $\mathcal{D}=\{\mathbf{x}_i\}_{i=1}^w$.

Now, we think like Bayesian once again and realize:

$$
P_B(\mathcal{D}|\mathcal{G}^0) = \int_{\boldsymbol{\Theta}_{\mathcal{G}^0}}
$$



